<a href="https://colab.research.google.com/github/SabeehAnsari/Pytorch-Image-classifier/blob/main/V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [30]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [31]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [32]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

In [33]:
image, label = train_data[0]

In [34]:
image.size()

torch.Size([3, 32, 32])

In [35]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [36]:
class NeuralNet(nn.Module):

  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3,12,5)  # (12,28,28)
    self.pool = nn.MaxPool2d(2,2)  # (12,14,14)
    self.conv2 = nn.Conv2d(12,24,5)
    self.fc1 = nn.Linear(24*5*5, 120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [37]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(30):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')


Loss: 1.0441
Training epoch 9...
Loss: 0.9980
Training epoch 10...
Loss: 0.9625
Training epoch 11...
Loss: 0.9239
Training epoch 12...
Loss: 0.8925
Training epoch 13...
Loss: 0.8676
Training epoch 14...
Loss: 0.8296
Training epoch 15...
Loss: 0.8040
Training epoch 16...
Loss: 0.7738
Training epoch 17...
Loss: 0.7489
Training epoch 18...
Loss: 0.7201
Training epoch 19...
Loss: 0.6974
Training epoch 20...
Loss: 0.6689
Training epoch 21...
Loss: 0.6471
Training epoch 22...
Loss: 0.6225
Training epoch 23...
Loss: 0.6009
Training epoch 24...
Loss: 0.5787
Training epoch 25...
Loss: 0.5592
Training epoch 26...
Loss: 0.5345
Training epoch 27...
Loss: 0.5157
Training epoch 28...
Loss: 0.4962
Training epoch 29...
Loss: 0.4750


In [38]:
torch.save(net.state_dict(), 'trained_net.pth')

In [39]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [42]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 9.98%


In [41]:
from PIL import Image
import torch
import torchvision.transforms as transforms

# Define the transformation steps for the input image
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize the image to match the input size for CIFAR10
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image
])

# Function to load and preprocess an image
def load_image(image_path):
    # Open the image
    image = Image.open(image_path)
    # Apply transformations (resize, convert to tensor, normalize)
    image = new_transform(image)
    # Add a batch dimension (1 for a single image)
    image = image.unsqueeze(0)
    return image

# Example image paths
image_paths = ['Airplane.jpg']

# Load and preprocess the images
images = [load_image(img) for img in image_paths]

# Set the model to evaluation mode
net.eval()

# Perform inference for each image
with torch.no_grad():
    for image in images:
        # Forward pass through the network
        output = net(image)
        # Get the predicted class
        _, predicted = torch.max(output, 1)
        # Print the predicted class name
        print(f'Prediction: {class_names[predicted.item()]}')


Prediction: cat
